In [1]:
# import sys
# sys.path.insert(0,"..")
# from broncode.cnn import CNN
# import torchvision.transforms as T
# from torchvision.io import read_image, ImageReadMode
# import matplotlib as plt


In [2]:
import torchvision.transforms as T
from torch.utils.data import Subset, Dataset, DataLoader, random_split
from torch import nn
import torch.nn.functional
from torchvision.datasets import ImageFolder
from torchvision.io import read_image, ImageReadMode


In [8]:
path = "../informatie/apple_disease_classification/images/Train/Dataset/good_apples/good_apple0.jpg"
img = read_image(path, ImageReadMode.UNCHANGED)/255

print(img.shape)
print(img)

torch.Size([3, 128, 128])
tensor([[[0.9059, 0.9059, 0.9059,  ..., 0.9294, 0.9294, 0.9294],
         [0.9059, 0.9059, 0.9059,  ..., 0.9294, 0.9294, 0.9294],
         [0.9059, 0.9059, 0.9059,  ..., 0.9294, 0.9294, 0.9294],
         ...,
         [0.9490, 0.9490, 0.9490,  ..., 0.9725, 0.9725, 0.9725],
         [0.9529, 0.9529, 0.9529,  ..., 0.9725, 0.9725, 0.9725],
         [0.9529, 0.9529, 0.9529,  ..., 0.9725, 0.9725, 0.9725]],

        [[0.9216, 0.9216, 0.9216,  ..., 0.9294, 0.9294, 0.9294],
         [0.9216, 0.9216, 0.9216,  ..., 0.9294, 0.9294, 0.9294],
         [0.9216, 0.9216, 0.9216,  ..., 0.9294, 0.9294, 0.9294],
         ...,
         [0.9412, 0.9412, 0.9412,  ..., 0.9529, 0.9529, 0.9529],
         [0.9451, 0.9451, 0.9451,  ..., 0.9529, 0.9529, 0.9529],
         [0.9451, 0.9451, 0.9451,  ..., 0.9529, 0.9529, 0.9529]],

        [[0.9569, 0.9569, 0.9569,  ..., 0.9686, 0.9686, 0.9686],
         [0.9569, 0.9569, 0.9569,  ..., 0.9686, 0.9686, 0.9686],
         [0.9569, 0.9569, 0.9569

In [10]:

dataset_path = "../informatie/apple_disease_classification/images/Train/Dataset/"
transform = T.ToTensor()
dataset = ImageFolder(dataset_path, transform=transform)

 

dataset.class_to_idx

{'bad_apples': 0, 'good_apples': 1}

In [11]:
# build subset
idx = [i for i in range(len(dataset)) if (dataset.imgs[i][1] == dataset.class_to_idx['bad_apples'] or dataset.imgs[i][1] == dataset.class_to_idx['good_apples']) ]
subset = Subset(dataset, idx)
 


for sample_idx in range(len(subset)):
    sample = subset[sample_idx]
    
    # Access the image and label from the sample
    image, label = sample
    
    # Print or examine the sample
    if sample_idx % 1000 == 0:
        print("Sample:", sample_idx,"Image shape:", image.shape,"Label:", label)

Sample: 0 Image shape: torch.Size([3, 128, 128]) Label: 0
Sample: 1000 Image shape: torch.Size([3, 128, 128]) Label: 0
Sample: 2000 Image shape: torch.Size([3, 128, 128]) Label: 0
Sample: 3000 Image shape: torch.Size([3, 128, 128]) Label: 0
Sample: 4000 Image shape: torch.Size([3, 128, 128]) Label: 0
Sample: 5000 Image shape: torch.Size([3, 128, 128]) Label: 0
Sample: 6000 Image shape: torch.Size([3, 128, 128]) Label: 0
Sample: 7000 Image shape: torch.Size([3, 128, 128]) Label: 0
Sample: 8000 Image shape: torch.Size([3, 128, 128]) Label: 0
Sample: 9000 Image shape: torch.Size([3, 128, 128]) Label: 0
Sample: 10000 Image shape: torch.Size([3, 128, 128]) Label: 0
Sample: 11000 Image shape: torch.Size([3, 128, 128]) Label: 0
Sample: 12000 Image shape: torch.Size([3, 128, 128]) Label: 0
Sample: 13000 Image shape: torch.Size([3, 128, 128]) Label: 0
Sample: 14000 Image shape: torch.Size([3, 128, 128]) Label: 0


In [12]:
counter = 0
bad_counter = 0
for i in range(len(subset)):
    if subset[i][1] == 1:
        counter += 1
    else:
        bad_counter += 1

print("good apples", counter)
print("bad apples", bad_counter)

good apples 208
bad apples 14544


In [15]:
generator1 = torch.Generator().manual_seed(13)

# create a train test split with 70% train, 30% test
train_dataset, test_dataset, val_dataset = random_split(subset, [0.6, 0.2, 0.2], generator=generator1)

# check length of train and test dataset
print(len(train_dataset), len(test_dataset), len(val_dataset))

# create train and test dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

8852 2950 2950


In [33]:
class CNN(nn.Module):
    
    def __init__(self, img_size=128, batch_size=64, c_in=3):
        super().__init__()

        self.model = nn.Sequential(
            nn.Conv2d(c_in, 16, 5, padding=2),
            nn.ReLU(),
            nn. Conv2d(16, 32, 5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 5, padding=2),
            nn.ReLU(),
            nn. Conv2d(64, 96, 5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Flatten(0),

            nn.Linear((img_size*768)*batch_size, 500),
            nn.ReLU(),
            nn.Linear(500, 256),
            nn.ReLU(),
            nn.Linear(256, 2),
            nn.Softmax(dim=0)
            )

    def fit(self, train_loader, val_loader, epochs, lr=0.001, opt_function=torch.optim.Adam, loss_func=nn.BCELoss):
        optimizer = opt_function(self.model.parameters(),lr )
        for epoch in range(epochs):
            
            for batch in train_loader:
                image, labels = batch
                pred = self(image)
                loss = loss_func(pred, labels)
                print(loss)


               

        


    def forward(self, data):

        return self.model(data)


In [34]:
net = CNN()
net.fit(train_loader, val_loader, epochs=1)